In [1]:
!git clone https://github.com/edwinkmusaasizi/Machine-Learning.git

fatal: destination path 'Machine-Learning' already exists and is not an empty directory.


In [2]:

!ls

Machine-Learning  sample_data


In [3]:
%cd Machine-Learning
!ls

/content/Machine-Learning
data	 Machine-Learning  notebooks  references  requirements.txt
LICENCE  models		   README.md  reports	  src


In [ ]:
!ls

cleaned_mental_health_data.csv


In [ ]:
import pandas as pd
df = pd.read_csv('cleaned_mental_health_data.csv')
print(df.head())

   age     sex   Religion       marital status education status residence  \
0   25    Male  christian               single         tertiary     urban   
1   55  Female  christian              widowed          primary     urban   
2   22    Male  christian  married /cohabiting        secondary     urban   
3   28    Male    Moslems               single          primary     urban   
4   33  Female  christian            separated          primary     rural   

   Income substance use  distance traveled in kilometers comorbidity  ...  \
0  300000           Yes                                2          No  ...   
1  243000            No                                3          No  ...   
2  200000           Yes                               20          No  ...   
3       1           Yes                                1          No  ...   
4       0            No                               25          No  ...   

  Menstrual or breast problem How long have you been on treatment (years) 

1 DATA PROCESSING

1.1 Define Target Variable

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Step 1: Inspect the relevant columns
print(df['Do you ever forget to take your medication?'].unique())
print(df['Are you careless at times about taking your medication?'].unique())
print(df['When you feel better, do you sometimes stop taking your medication?'].unique())
print(df['Sometimes if you feel worse when you take the medication, do you stop taking it?'].unique())

# Step 2: Define the target variable
df['adherence'] = (
    (df['Do you ever forget to take your medication?'] == 'No') &
    (df['Are you careless at times about taking your medication?'] == 'No') &
    (df['When you feel better, do you sometimes stop taking your medication?'] == 'No') &
    (df['Sometimes if you feel worse when you take the medication, do you stop taking it?'] == 'No')
).astype(int)

# Step 3: Verify the target variable
print(df['adherence'].value_counts())
sns.countplot(data=df, x='adherence')
plt.title('Distribution of Adherence')
plt.show()

# Step 4: Drop the original columns (to avoid redundancy)
df = df.drop(columns=[
    'Do you ever forget to take your medication?',
    'Are you careless at times about taking your medication?',
    'When you feel better, do you sometimes stop taking your medication?',
    'Sometimes if you feel worse when you take the medication, do you stop taking it?'
])

# Display the first few rows after processing
print(df.head())

KeyError: 'Do you ever forget to take your medication?'

1.2 Select Features

In [ ]:
# Step 1: Drop irrelevant columns
X = df.drop(columns=[
    'adherence',  # Target variable
    'If you have any further comments about medication or this questionnaire, please write them below'  # Free-text column
])

# Step 2: Separate the target variable
y = df['adherence']

# Step 3: Verify the features and target
print("Shape of X (features):", X.shape)
print("Shape of y (target):", y.shape)

# Step 4: Inspect the features
print(X.columns)

Shape of X (features): (117, 60)
Shape of y (target): (117,)
Index(['age', 'sex', 'Religion', 'marital status', 'education status',
       'residence', 'Income', 'substance use',
       'distance traveled in kilometers', 'comorbidity',
       'I take my medication only when I am sick',
       'It is unnatural for my mind and body to be controlled by medication?',
       'My thoughts are clearer on medication',
       'By staying on medication, I can prevent getting sick',
       'I feel weird, like a ‘zombie’ on medication',
       'Medication makes me feel tired and sluggish',
       'Some of your symptoms are made by your mind.', 'You are mentally well',
       'You do not need medication', 'Your stay in the hospital is necessary',
       'The doctor is right in prescribing medication for you.',
       'You do not need to be seen by a doctor or psychiatrist',
       'If someone said you have a nervous or mental illness, they would be right',
       'None of the unusual things you are

**1.3 Encoding Categorical Values**

In [ ]:
# Step 1: Identify categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns
print("Categorical columns:", categorical_columns)

# Step 2: Perform one-hot encoding
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Step 3: Verify the encoded features
print("Column names after encoding:", X.columns)
print("Shape of X after encoding:", X.shape)

# Display the first few rows after encoding
print(X.head())

Categorical columns: Index([], dtype='object')
Column names after encoding: Index(['age', 'Income', 'distance traveled in kilometers',
       'For me, the good things about medication outweigh the bad',
       'I feel strange, "doped up", on medication',
       'I take medications of my own free choice',
       'Medications make me feel more relaxed',
       'Medication makes me feel tired and sluggish.1',
       'I take medication only when I feel ill',
       'I feel more normal on medication',
       'It is unnatural for my mind and body to be controlled by medications',
       'My thoughts are clearer on medication.1',
       'Taking medication will prevent me from having a breakdown',
       'How long have you been on treatment (years)',
       'How satisfied are you with the psychiatry medications you are taking',
       'How many pills do you swallow per day',
       'I get social support from my family and friends about my mental illness',
       'sex_Male', 'Religion_christian

**1.4 spliting the Data**
We’ll split the dataset into:

Training set: 80% of the data (used to train the model).

Testing set: 20% of the data (used to evaluate the model).

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Verify the split
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (93, 82)
Shape of X_test: (24, 82)
Shape of y_train: (93,)
Shape of y_test: (24,)


2 Training Evaluating A Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import time

# Step 1: Initialize the model
model = RandomForestClassifier(random_state=42)

# Step 2: Train the model and measure training time
start_time = time.time()  # Start timer
model.fit(X_train, y_train)
training_time = time.time() - start_time  # End timer

# Step 3: Make predictions
y_pred = model.predict(X_test)

# Step 4: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Calculate specificity and sensitivity
TN = conf_matrix[0, 0]  # True Negatives
FP = conf_matrix[0, 1]  # False Positives
FN = conf_matrix[1, 0]  # False Negatives
TP = conf_matrix[1, 1]  # True Positives

specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)

print("Specificity:", specificity)
print("Sensitivity (Recall):", sensitivity)

# Display training time
print("Training Time:", training_time, "seconds")

# Step 5: Analyze feature importance
importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("Top 10 Features by Importance:\n", importance_df.head(10))

Accuracy: 0.6666666666666666
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.88      0.78        16
           1       0.50      0.25      0.33         8

    accuracy                           0.67        24
   macro avg       0.60      0.56      0.56        24
weighted avg       0.63      0.67      0.63        24

Confusion Matrix:
 [[14  2]
 [ 6  2]]
Specificity: 0.875
Sensitivity (Recall): 0.25
Training Time: 0.14958524703979492 seconds
Top 10 Features by Importance:
                                               Feature  Importance
13        How long have you been on treatment (years)    0.065305
0                                                 age    0.061948
2                     distance traveled in kilometers    0.055137
14  How satisfied are you with the psychiatry medi...    0.047528
29       I take my medication only when I am sick_Yes    0.044658
16  I get social support from my family and friend...    0.034237